# **Attitude sensitivity analysis**

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from tqdm.notebook import tqdm

In [ ]:
class Buyer:
    def __init__(self, buyer_id, distances, pv=1000, beta=0.5, pvmax=1000):
        self.id = buyer_id
        self.state = "overbid"
        self.pv = pv
        self.beta = beta
        self.pvmax = pvmax
        self.distances = distances
        self.prices = {}
        
    def inform_price(self, auction_id, price):
        self.prices[auction_id] = price if price <= self.pv else np.inf
        if np.all(np.array(self.prices) == np.inf):
            self.state = "out_of_budget"
        
    def compute_costs(self):
        prices, distances, id_list = [], [], []
        for p in self.distances:
            prices.append(self.prices[p])
            distances.append(self.distances[p])
            id_list.append(p)
        prices = np.array(prices)
        distances = np.array(distances)
        cost = self.beta*prices/self.pvmax + (1-self.beta)*distances/np.max(distances)
        return cost, id_list
        
    def ask_bid(self, auction_id, price):
        bids = False
        if (self.state == "overbid") and (price <= self.pv):
            costs, ids = self.compute_costs()
            prefs = np.where(costs == costs.min())[0]
            pref_ids = []
            for p in prefs:
                pref_ids.append(ids[p])
            bids = auction_id in pref_ids
        elif price > self.pv:
            self.prices[auction_id] = np.inf
        if bids:
            self.state = "winning"
        return bids
    
    def tell_overbid(self):
        self.state = "overbid"
        
    def tell_won(self):
        self.state = "won"

In [ ]:
class Auction:
    def __init__(self, id_, starting_price, epsilon=1.0):
        self.id = id_
        self.price = starting_price
        self.buyers = []
        self.epsilon = epsilon
        self.winner = None
        
    def add_buyer(self, buyer):
        self.buyers.append(buyer)
        
    def _inform_buyers(self):
        for b in self.buyers:
            b.inform_price(self.id, self.price)
        
    def auction_round(self):
        bid_received = False
        for b in self.buyers:
            bids = b.ask_bid(self.id, self.price)
            if bids:
                if self.winner is not None:
                    self.winner.tell_overbid()
                self.winner = b
                self.price += self.epsilon
                self._inform_buyers()
                bid_received = True
        return bid_received
                
    def terminate(self):
        winner_id = ""
        if self.winner is not None:
            self.winner.tell_won()
            winner_id = self.winner.id
        return {"winner": winner_id,
                "price" : self.price-self.epsilon}

In [ ]:
def check_terminable(buyers):
    non_terminated_buyers = 0
    for b in buyers:
        if (b.state != "winning") and (b.state != "out_of_budget"):
            non_terminated_buyers += 1
    return len(buyers) - non_terminated_buyers

def run_auctions_with_prices(auctions, buyers, r_max):
    auction_results = {}
    auction_prices = []
    rs = np.zeros(len(auctions))
    
    #pbar = tqdm(total=len(buyers))
    terminables = 0
    while np.sum(rs<r_max) and (terminables != len(buyers)):
        ap_ = {}
        for a in auctions:
            ap_[a.id] = a.price
        auction_prices.append(ap_)
        
        #pbar.n = terminables
        #pbar.refresh()
        for i, a in enumerate(auctions):
            bid_received = a.auction_round()
            if bid_received:
                rs[i] = 0
            else:
                rs[i] += 1
        terminables = check_terminable(buyers)
                
    #pbar.n = len(auctions)
    #pbar.refresh()
    for a in auctions:
        auction_results[a.id] = a.terminate()  
    #pbar.close()
    return auction_results, auction_prices

def run_auctions(auctions, buyers, r_max):
    auction_results = {}
    rs = np.zeros(len(auctions))
    
    #pbar = tqdm(total=len(auctions))
    terminables = 0
    while np.sum(rs<r_max) and (terminables != len(buyers)):
        #pbar.n = terminables
        #pbar.refresh()
        for i, a in enumerate(auctions):
            bid_received = a.auction_round()
            if bid_received:
                rs[i] = 0
            else:
                rs[i] += 1
        terminables = check_terminable(buyers)
                
    #pbar.n = len(auctions)
    #pbar.refresh()
    for a in auctions:
        auction_results[a.id] = a.terminate()  
    #pbar.close()
    return auction_results

In [ ]:
def get_won_auctions(auction_results, buyers):
    won_auctions = {}
    for b in buyers:
        for a in auction_results:
            if auction_results[a]["winner"] == b.id:
                won_auctions[b.id] = a
    return won_auctions

In [ ]:
starting_prices = [1.09, 0.73, 0.49, 0.36, 0.24, 0.12]
distances = [0, 100, 141, 200, 224, 283]

In [ ]:
winner_betas, winner_destinations, winner_prices = [],[],[]

for i in range(100000):
    betas = np.random.uniform(0.1, 0.9, 10)
    validations = np.random.uniform(.66, 1.50, 10)
    
    if i%100==0: print(f"round {i}")
    
    buyers = []
    for b_i in range(10):
        distance_ = {}
        for a_i in distances:
            distance_[f"a_{a_i}"] = a_i
        buyers.append(Buyer(f"b_{b_i}", distance_, beta=betas[b_i], pv=validations[b_i], pvmax=1.5))
    auctions = [Auction(f"a_{distances[i]}", starting_prices[i], epsilon=.01) for i in range(6)]
    for a in auctions:
        for b in buyers:
            a.add_buyer(b)
            b.inform_price(a.id, a.price)
            
    results = run_auctions(auctions, buyers, 1)
    
    for r in results:
        if results[r]["winner"] != "":
            winner_destinations.append(int(r.split("_")[-1]))
            winner_idx = int(results[r]["winner"].split("_")[-1])
            winner_betas.append(betas[winner_idx])
            winner_prices.append(results[r]["price"])

In [ ]:
auction_results = {
    "winner_betas": winner_betas,
    "winner_destinations": winner_destinations,
    "winner_prices": winner_prices
}

In [ ]:
import json
with open("../02_data/sensitivity.json", "w") as f:
    json.dump(auction_results, f)

In [ ]:
ar_df = pd.DataFrame.from_dict(auction_results)

In [ ]:
avg_results = ar_df.groupby("winner_destinations").mean()

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(12,4))
ln1 = ax.plot(avg_results.index, avg_results["winner_betas"], label="avg. $\\beta$")
ax.scatter(avg_results.index, avg_results["winner_betas"], label="avg. $\\beta$")
bx = ax.twinx()
ln2 = bx.plot(avg_results.index, avg_results["winner_prices"], color="tab:orange", label="avg. price [€]")
bx.scatter(avg_results.index, avg_results["winner_prices"], color="tab:orange")

plt.xticks(distances)
ax.set_xlabel("distance of parking lots\nfrom the center [m]")
ax.set_ylabel("winner's $\\beta$")
bx.set_ylabel("winning price [€]")

lns = ln1+ln2
labs = [l.get_label() for l in lns]
ax.legend(lns, labs, loc="upper center")

In [ ]:
ar_df["beta_q"] = pd.cut(ar_df["winner_betas"], bins = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
                        labels = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])
avg_results2 = ar_df.groupby("beta_q").mean()

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(12,4))
ln1 = ax.plot(avg_results2.index, avg_results2["winner_destinations"], label="avg. distances [m]")
ax.scatter(avg_results2.index, avg_results2["winner_destinations"])
bx = ax.twinx()
ln2 = bx.plot(avg_results2.index, avg_results2["winner_prices"], color="tab:orange", label="avg. price [€]")
bx.scatter(avg_results2.index, avg_results2["winner_prices"], color="tab:orange")

ax.set_ylabel("distance of parking lots\nfrom the center [m]")
ax.set_xlabel("winner's $\\beta$")
bx.set_ylabel("winning price [€]")

lns = ln1+ln2
labs = [l.get_label() for l in lns]
ax.legend(lns, labs, loc="center left")